# 0.5 Generate Noisy Data and Fitting

This notebook is used to generate and fit all the data required for the paper. We will generate data for the following noise cases: 

1, 2, 3, 4, 5, 6, 7 STD

## Imports

In [1]:
import sys

sys.path.append("../../")
# sys.path.append("/home/ferroelectric/Documents/m3_learning/m3_learning/src")
sys.path.append('../../src')


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from m3_learning.be.dataset import BE_Dataset
from m3_learning.viz.printing import printer
from m3_learning.be.nn import SHO_fit_func_nn, SHO_Model
from m3_learning.util.file_IO import download_and_unzip


2024-06-24 05:31:17.653229: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 05:31:18.185943: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/cv2/../../lib64:
2024-06-24 05:31:18.185989: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ferroelectric/micromamba/envs/paper/

## Loading data for SHO fitting


In [3]:
# Download the data file from Zenodo
url = 'https://zenodo.org/record/7774788/files/PZT_2080_raw_data.h5?download=1'

# Specify the filename and the path to save the file
filename = '/data_raw_.h5'
save_path = './Data'

# download the file
download_and_unzip(filename, url, save_path)

Using files already downloaded


In [4]:
data_path = "/home/ferroelectric/Documents/m3_learning/m3_learning/src/m3_learning/tests/be/build_test.h5"  # TEST_DATA.h5"

# instantiate the dataset object
dataset = BE_Dataset(data_path)

# print the contents of the file
dataset.print_be_tree()

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
Datasets and datagroups within the file:
------------------------------------
/
/Measurement_000
/Measurement_000/Channel_000
/Measurement_000/Channel_000/Bin_FFT
/Measurement_000/Chann

## Generates Noisy Data

This function will generate noisy records and save them as an h5_main file in the USID format. This allows the data to be computed with the Pycroscopy SHO Fitter. 

In [5]:
# calculates the standard deviation and uses that for the noise
noise_STD = np.std(dataset.get_original_data)

# prints the standard deviation
print(noise_STD)

0.020149399


In [6]:
dataset.generate_noisy_data_records(noise_levels = np.arange(1,2), 
                                    verbose=True, 
                                    # basegroup='/',
                                    noise_STD=noise_STD)

The STD of the data is: 0.02014939859509468
Adding noise level 1


## SHO fits on all the datasets

This will take some time, Each fit takes about 10 minutes to complete. 

In [7]:
out = [f"Noisy_Data_{i}" for i in np.arange(1,2)]
out.append("Raw_Data")

for data in out:
    print(f"Fitting {data}")
    dataset.SHO_Fitter(dataset = data, h5_sho_targ_grp = f"{data}_SHO_Fit", max_mem=1024*64, max_cores= 20)

Fitting Noisy_Data_1
Working on:
/home/ferroelectric/Documents/m3_learning/m3_learning/src/m3_learning/tests/be/build_test.h5
['Y', 'X'] [10, 10]


SHO Fits will be written to:
/home/ferroelectric/Documents/m3_learning/m3_learning/src/m3_learning/tests/be/build_test.h5


Consider calling test() to check results before calling compute() which computes on the entire dataset and writes results to the HDF5 file
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 finished parallel computation
Rank 0 - 100% complete. Time remaining: 0.0 msec
Finished processing the entire dataset!

Note: SHO_Fit has already been performed with the same parameters before. These results will be returned by compute() by default. Set override to True to force fr

/home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Rank 0 - 100% complete. Time remaining: 0.0 msec
Finished processing the entire dataset!
LSQF method took 130.51411652565002 seconds to compute parameters
Fitting Raw_Data
Working on:
/home/ferroelectric/Documents/m3_learning/m3_learning/src/m3_learning/tests/be/build_test.h5
['Y', 'X'] [10, 10]


SHO Fits will be written to:
/home/ferroelectric/Documents/m3_learning/m3_learning/src/m3_learning/tests/be/build_test.h5


Consider calling test() to check results before calling compute() which computes on the entire dataset and writes results to the HDF5 file
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 finished parallel computation
Rank 0 - 100% complete. Time remaining: 0.0 msec
Finished processing the entire dataset!

Note: SHO_F

/home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Rank 0 - 100% complete. Time remaining: 0.0 msec
Finished processing the entire dataset!
LSQF method took 101.46816372871399 seconds to compute parameters


### Checks the results to make sure it was saved correctly

In [8]:
# print the contents of the file
dataset.print_be_tree()

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Noisy_Data_1
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
├ Noisy_Data_1_SHO_Fit
  --------------------
  ├ Noisy_Data_1-SHO_Fit_000
    ------------------------
    ├ Fit
    ├ Guess
    ├ Spectroscopic_Indices
    ├ Spectr